In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
!pip install gensim
!pip install imbalanced-learn

In [2]:
from sklearn.pipeline import Pipeline
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import set_config
from sklearn.metrics import confusion_matrix

set_config(display='diagram')

In [3]:
from transformers import RemoveWords, GetTags, GetBase, Limpieza, \
                        common_tag_list, Tokenize, UnTokenize, all_tag_list
from transformers import en_embeddings_subset, GetSentenceEmbedding

In [4]:
np.random.seed(42)

In [5]:
data = pd.read_csv('datasets/dontpatronizeme_pcl.tsv', skiprows=4, sep='\t', 
                   header=None, names=['index','id_new','word_key', 'country','text','label'])
mapeo = lambda x : 0 if x < 2 else 1 
data['label'] = data['label'].apply(mapeo)
data.head()

,index,id_new,word_key,country,text,label
0,5,@@1494111,refugee,ca,""" Just like we received migrants fleeing El Sa...",0
1,6,@@9382277,in-need,in,"To bring down high blood sugar levels , insuli...",0
2,7,@@7562079,refugee,za,The European Union is making an historic mista...,0
3,8,@@23663488,hopeless,nz,""" They 're either hopeless for being beaten by...",0
4,9,@@3449225,homeless,ph,"NUEVA ERA , Ilocos Norte - No family shall be ...",0


array(['refugee', 'in-need', 'hopeless', 'homeless', 'migrant',
       'vulnerable', 'women', 'poor-families', 'disabled', 'immigrant'],
      dtype=object)

In [28]:
pipe_pre = Pipeline(steps=[('limpieza', Limpieza()),
                           ('tokenize', Tokenize()),
                           ('remove_stopwords', RemoveWords()),
                           ('obtener_tags', GetTags(tag_list = common_tag_list)),
                           ('get_base', GetBase()),
                           ('untokenize', UnTokenize()),
                           ('word2vec', GetSentenceEmbedding())
                          ])

In [29]:
data = data[~data.text.isna()]
X = pipe_pre.fit_transform(data['text'])
y = data['label']

c:\Users\anapm\Documents\GitHub\Patronizing_and_Condescending_Language_Detection\transformers\limpieza.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  return pd.Series(X).str.replace(" n\'t", "n\'t") \
c:\Users\anapm\Documents\GitHub\Patronizing_and_Condescending_Language_Detection\transformers\limpieza.py:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  return pd.Series(X).str.replace(" n\'t", "n\'t") \


Series([], Name: text, dtype: object)


In [8]:
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN


In [30]:
X_train , X_test, y_train, y_test = train_test_split(X,y)

In [31]:
y_test.value_counts()

0    2384
1     232
Name: label, dtype: int64

In [32]:
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [38]:
oversample = SMOTE()
X_test, y_test = oversample.fit_resample(X_test, y_test)

In [29]:
y_train.value_counts()

0    7089
1    7089
Name: label, dtype: int64

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

In [34]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
model = RandomForestClassifier()
cv_results = cross_validate(model, X_train, y_train, scoring='roc_auc', cv=cv, 
                            n_jobs=-1,
                           return_estimator = True)

In [35]:
idx = np.argmax(cv_results['test_score'])
model = cv_results['estimator'][idx]

In [39]:
labels = model.predict(X_test)
report = classification_report(y_test, labels)
print(report)

              precision    recall  f1-score   support

           0       0.55      0.95      0.70      2384
           1       0.82      0.23      0.37      2384

    accuracy                           0.59      4768
   macro avg       0.69      0.59      0.53      4768
weighted avg       0.69      0.59      0.53      4768



In [16]:
from sklearn.metrics import f1_score

In [40]:
f1_score(y_test, labels)

0.365296803652968

In [24]:
(0.94 + .21)/2

0.575

# Aqui no

In [23]:

pipe1 = Pipeline(steps=[('limpieza', Limpieza()),
                         ('tokenize', Tokenize()),
                        ('remove_stopwords', RemoveWords()),
                        ('obtener_tags', GetTags(tag_list = common_tag_list)),
                         ('get_base', GetBase()),
                         ('untokenize', UnTokenize()),
                         ('tfidf', GetSentenceEmbedding()),
                          ('svm', SVC())
                         ])
res = pipe1.fit(X,y)

Series([], Name: text, dtype: object)


KeyboardInterrupt: 

In [ ]:
res.score(X, y)

In [ ]:
X_train , X_test, y_train, y_test = train_test_split(X,y)

In [ ]:
clf = pipe1.fit(X_train,y_train)

In [ ]:
labels = clf.predict(X_test)

In [5]:
labels = clf.predict(X_test)
report = classification_report(y_test, labels)
print(report)

NameError: name 'y_test' is not defined

In [11]:
report = classification_report(y_test, labels)
print(report)

              precision    recall  f1-score   support

           0       0.91      1.00      0.95      2378
           1       0.00      0.00      0.00       238

    accuracy                           0.91      2616
   macro avg       0.45      0.50      0.48      2616
weighted avg       0.83      0.91      0.87      2616



C:\Users\anapm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\anapm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\anapm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
confusion_matrix(y_test, labels)

array([[2377,    2],
       [ 236,    1]], dtype=int64)

In [27]:
sum(labels)

3

In [30]:
2377/(2377+236)

0.9096823574435515